<a href="https://colab.research.google.com/github/anastasiiakucherenko/LLM-playground/blob/main/Fine_tuning_HFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fine-tuning

Here I fine-tune model on IMBD dataset of sentiment for movie reviews.
First, we download the dataset.


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("stanfordnlp/imdb")
checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
raw_datasets

In [ ]:
raw_datasets['train'] = raw_datasets['train'].shuffle(seed=42).select(range(1000))
raw_datasets['test'] = raw_datasets['test'].shuffle(seed=42).select(range(200))
raw_datasets['validate'] = raw_datasets['test'].shuffle(seed=37).select(range(200))

In [ ]:
train_labels = raw_datasets['train']['label']
print("Unique labels:", set(train_labels))
print("Label counts:", {label: train_labels.count(label) for label in set(train_labels)})

In [ ]:
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
train_dataset = raw_datasets["train"]
print(train_dataset[2:5])

Now, we proceed to describing the model and fine-tuning. We start with just using standard Trainer.

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    processing_class=tokenizer,
)

In [ ]:
trainer.train()

In [ ]:
# Get some test examples
test_examples = raw_datasets["test"].select(range(5))  # First 5 test examples

# Make predictions
predictions = trainer.predict(tokenized_datasets["test"].select(range(5)))
predicted_labels = predictions.predictions.argmax(-1)

# Print results
for i, example in enumerate(test_examples):
    actual_label = "Positive" if example["label"] == 1 else "Negative"
    predicted_label = "Positive" if predicted_labels[i] == 1 else "Negative"

    print(f"Review: {example['text'][:100]}...")  # First 100 characters
    print(f"Actual: {actual_label}")
    print(f"Predicted: {predicted_label}")
    print("-" * 50)

In [ ]:
predictions = trainer.predict(tokenized_datasets["validate"])
print(predictions.predictions.shape, predictions.label_ids.shape)


In [ ]:
import numpy as np
preds = np.argmax(predictions.predictions, axis=-1)

# Load multiple metrics for binary classification
accuracy = evaluate.load("accuracy")
precision = evaluate.load("precision")
recall = evaluate.load("recall")
f1 = evaluate.load("f1")

# Compute all metrics
results = {
    "accuracy": accuracy.compute(predictions=preds, references=predictions.label_ids),
    "precision": precision.compute(predictions=preds, references=predictions.label_ids),
    "recall": recall.compute(predictions=preds, references=predictions.label_ids),
    "f1": f1.compute(predictions=preds, references=predictions.label_ids)
}

print(results)

Now I try different training_args and retrain to compare the results

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",

    # Learning rate tuning
    learning_rate=3e-5,  # Try 1e-5, 2e-5, 3e-5, 5e-5
    lr_scheduler_type="cosine",  # Try "linear", "cosine", "polynomial"

    # Training dynamics
    num_train_epochs=3,  # Try 3-10 epochs
    warmup_ratio=0.1,    # Warm up learning rate
    weight_decay=0.01,   # Regularization

    # Batch size (adjust based on GPU memory)
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,

    # Early stopping and saving
    #load_best_model_at_end=True,
    #metric_for_best_model="eval_accuracy",
    #greater_is_better=True,
)


In [1]:
from transformers import EarlyStoppingCallback

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    processing_class=tokenizer
)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipython-input-112497865.py", line 1, in <cell line: 0>
    from transformers import EarlyStoppingCallback
  File "/usr/local/lib/python3.11/dist-packages/transformers/__init__.py", line 27, in <module>
    from . import dependency_versions_check
  File "/usr/local/lib/python3.11/dist-packages/transformers/dependency_versions_check.py", line 16, in <module>
    from .utils.versions import require_version, require_version_core
  File "/usr/local/lib/python3.11/dist-packages/transformers/utils/__init__.py", line 24, in <module>
    from .auto_docstring import (
  File "/usr/local/lib/python3.11/dist-packages/transformers/utils/auto_docstring.py", line 30, in <module>
    from .generic import ModelOutput
  File "/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py", li

TypeError: object of type 'NoneType' has no len()

In [ ]:
trainer.train()

In [ ]:
predictions = trainer.predict(tokenized_datasets["validate"])
print(predictions.predictions.shape, predictions.label_ids.shape)


preds = np.argmax(predictions.predictions, axis=-1)



In [ ]:
# Load multiple metrics for binary classification
accuracy = evaluate.load("accuracy")
precision = evaluate.load("precision")
recall = evaluate.load("recall")
f1 = evaluate.load("f1")

# Compute all metrics
results = {
    "accuracy": accuracy.compute(predictions=preds, references=predictions.label_ids),
    "precision": precision.compute(predictions=preds, references=predictions.label_ids),
    "recall": recall.compute(predictions=preds, references=predictions.label_ids),
    "f1": f1.compute(predictions=preds, references=predictions.label_ids)
}

print(results)